# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

In [1]:
import pandas as pd 

## Откройте и изучите файл

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
display(data.head(10))

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
data.describe().round(2)

,calls,minutes,messages,mb_used,is_ultra
count,3214.00,3214.00,3214.00,3214.00,3214.00
mean,63.04,438.21,38.28,17207.67,0.31
std,33.24,234.57,36.15,7570.97,0.46
min,0.00,0.00,0.00,0.00,0.00
25%,40.00,274.58,9.00,12491.90,0.00
50%,62.00,430.60,30.00,16943.24,0.00
75%,82.00,571.93,57.00,21424.70,1.00
max,244.00,1632.06,224.00,49745.73,1.00


## Разбейте данные на выборки

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
features = data.drop('is_ultra', axis=1)
target = data['is_ultra']

In [8]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4,train_size=0.6, random_state=123)

features_test, features_valid, target_test, target_valid = train_test_split(features_valid,target_valid,test_size = 0.25) 


## Исследуйте модели

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Начнем с модели дерева решений: исследуем некоторую глубину моделей

In [10]:
best_model = None
best_result = 0
for depth in range(1, 6):
    decision_tree_model = DecisionTreeClassifier(random_state=123, max_depth= depth)
    decision_tree_model.fit(features_train, target_train) 
    predictions = decision_tree_model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        best_model = decision_tree_model
        best_result = result
        
print("Accuracy лучшей модели:", best_result)
print('Характеристика лучшей модели:', '\n', best_model)

Accuracy лучшей модели: 0.8043478260869565
Характеристика лучшей модели: 
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')


Теперь разберемся с RandomForest. Здесь рассмотрим количество деревьев

In [11]:
best_model = None
best_result = 0
for est in range(10, 20, 50):
    RandomForest_model = RandomForestClassifier(random_state=123, n_estimators= est) 
    RandomForest_model.fit(features_train, target_train) 
    result = RandomForest_model.score(features_valid, target_valid)
    if result > best_result:
        best_model = RandomForest_model 
        best_result = result
print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print('Характеристика лучшей модели:', '\n', best_model)

Accuracy наилучшей модели на валидационной выборке: 0.782608695652174
Характеристика лучшей модели: 
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)


Поскольку стоит задача классификации теперь надо рассмотреть модель логистической регресси. 

In [12]:
LogisticRegression_model = LogisticRegression(random_state=123)
LogisticRegression_model.fit(features_train, target_train)
print('Accuracy',LogisticRegression_model.score(features_valid, target_valid))

Accuracy 0.6863354037267081


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Такая точность не подходит под ТЗ

## Проверьте модель на тестовой выборке

Результат модели дерева решений на тестовой выборке:

In [16]:
decision_tree_model.score(features_test, target_test)

0.7977178423236515

Данная модель уже удовлетворяет условиям технического задания. Попробуем улучшить результат с другими моделями.

Результат модели Random Forest на тестовой выборке:

In [17]:
RandomForest_model.score(features_test, target_test)

0.8008298755186722

И наконец логистическая регрессия:

In [27]:

LogisticRegression_model.score(features_test, target_test)

0.7157676348547718

## (бонус) Проверьте модели на адекватность

Тепрь еще раз оценим каждую модель, но уже по нескольким другим метрикам из инструментария SKLearn 

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

In [25]:
print('accuracy:', accuracy_score(target_test, decision_tree_model.predict(features_test)))
print('roc_auc:', roc_auc_score(target_test, decision_tree_model.predict(features_test)))
print('f1 score:', f1_score(target_test, decision_tree_model.predict(features_test),average='weighted'))

accuracy: 0.7977178423236515
roc_auc: 0.7094555298792068
f1 score: 0.7839116956966985


In [26]:
print('accuracy:', accuracy_score(target_test, RandomForest_model.predict(features_test)))
print('roc_auc:', roc_auc_score(target_test, RandomForest_model.predict(features_test)))
print('f1 score:', f1_score(target_test, RandomForest_model.predict(features_test),average='weighted'))

accuracy: 0.8008298755186722
roc_auc: 0.7287325204967602
f1 score: 0.7925202602552947


In [28]:
print('accuracy:', accuracy_score(target_test, LogisticRegression_model.predict(features_test)))
print('roc_auc:', roc_auc_score(target_test, LogisticRegression_model.predict(features_test)))
print('f1 score:', f1_score(target_test, LogisticRegression_model.predict(features_test),average='weighted'))

accuracy: 0.7157676348547718
roc_auc: 0.5316805541973968
f1 score: 0.6239569945597264


## Вывод

Итак, показатели моделей DecisionTree и RandomForest достаточно успешно прошли тесты другими метриками, а вот логистическая регрессия неплохо показавшая себя на тестовой выборке совсем провалила остальные "зачеты".